Setting up and importing:

In [1]:
# Lightning related imports

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning/
!pip install torchtext==0.8.0 torch==1.7.0 torchvision==0.8 pytorch-lightning==1.2.2
!pip install neptune-client==0.5.1

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
# https://github.com/PyTorchLightning/pytorch-lightning/issues/6210

     |████████████████████████████████| 7.0MB 12.3MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 11.8MB 47.9MB/s 
     |████████████████████████████████| 819kB 40.8MB/s 
     |████████████████████████████████| 276kB 47.4MB/s 
     |████████████████████████████████| 112kB 55.3MB/s 
     |████████████████████████████████| 1.3MB 47.7MB/s 
     |████████████████████████████████| 296kB 57.2MB/s 
     |████████████████████████████████| 143kB 55.9MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=f5d60fda6ffcb67bb5792074fe7a1551a189157dc5eaff52da22c653516353d3
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
ERROR: pytorch-lightning 1.2.2 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:


In [2]:
# !pip install torch==1.7.0 torchvision==0.8 torchtext==0.8
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric
!pip install ogb

     |████████████████████████████████| 2.8MB 24.5MB/s 
     |████████████████████████████████| 1.6MB 27.2MB/s 
     |████████████████████████████████| 194kB 18.7MB/s 
     |████████████████████████████████| 235kB 43.1MB/s 
     |████████████████████████████████| 2.2MB 55.1MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
  Created wheel for outdated: filename=outdated-0.2.0-cp37-none-any.whl size=4962 sha256=683c17c5bac8647797d363a255bf16c8d2e97a772cb52b44e7f956c2c5b8dc56
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp37-none-any.whl size=7051 sha256=96f2b54148a9c813d798ac8c73793ac52f7163728d4546a9c3234e591faeb700
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built outdated littleutils


In [3]:
%load_ext autoreload
%autoreload 2
import pytorch_lightning
print(pytorch_lightning.__version__)
from pytorch_lightning import Trainer
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

import json
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from scikitplot.metrics import plot_confusion_matrix
from google.colab import files
from lightning_model import LightningPAN


1.2.2


In [4]:
# Testing if the lightning_model script gets refreshed correctly
LightningPAN.training_epoch_end("hello",'Hle')


basemodel-nopoolend-posweight-meanloss


TypeError: ignored

In [5]:

# loading params
with open('parameters.json') as json_file:
    parameters = json.load(json_file)

print(parameters)


datasetname = parameters["dataset_name"]
phi = parameters["phi"]
runs = parameters["runs"]
batch_size = parameters["batch_size"]
filter_size = parameters["maximum_path_size"]+1
learning_rate = parameters["learning_rate"]
weight_decay = parameters["weight_decay"]
pool_ratio = parameters["pool_ratio"] # fraction of nodes to use for pooled feature array
pan_pool_weight = parameters['pan_pool_weight'] # importance weight on global matrix X
criterion_pos_weight = parameters['criterion_pos_weight'] # Over-weightage on positive labels 
nhid = parameters["nhid"]
epochs = parameters['epochs']

# Number of iterations to run model
num_iters = 5


{'dataset_name': 'ogbg-molhiv', 'phi': 0.3, 'runs': 1, 'num_workers': 0, 'batch_size': 128, 'maximum_path_size': 3, 'learning_rate': 0.001, 'weight_decay': 0.0005, 'pool_ratio': 0.5, 'pan_pool_weight': 0.5, 'criterion_pos_weight': 30, 'nhid': 512, 'epochs': 150}


Running the model 5 times and saving model checkpoints by epochs:

In [6]:
exp_id_dict = {} 
for i in range(num_iters):
  print("***********************RUNNING MODEL FOR ITERATION " + str(i) + '*********************')
  # LearningRateMonitor_Params = {'logging_interval': 'epoch'}
  # lr_logger = LearningRateMonitor(**LearningRateMonitor_Params)

  # Create model checkpoint directory
  os.system("mkdir models_"+str(i))

  checkpoint_callback = ModelCheckpoint(
      monitor='validation_loss',
      save_top_k=-1, save_last=True, save_weights_only=True,
      mode='min',
      dirpath='/content/models_'+ str(i) + '/',
      filename='PAN-{epoch:02d}-{validation_loss:.4f}-{rocauc_eval:.4f}',
      period=1
  )

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  print('Device: {}'.format(device))

  model = LightningPAN(9, 1, nhid=nhid, 
                      ratio=pool_ratio, pan_pool_weight=pan_pool_weight,
                      criterion_pos_weight=criterion_pos_weight,
                      filter_size=filter_size)
  neptune_logger = NeptuneLogger(
                  api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZDMyZTE4YmUtZTU1Yy00MDhhLTgyMzMtOTJiNzJjMTdkYWJkIn0=",
                  project_name='leyldy/drugs-PAN-baseline',
                  close_after_fit=False,
                  params=parameters, # your hyperparameters, immutable
                  tags=['baseline model', 'filter size across all layers', "no3rdPool", "iter " + str(i)],  # tags
                  upload_source_files=["parameters.json", "lightning_model.py"]
                  )
  
  exp_id_dict[neptune_logger.experiment.id] = i

  trainer = Trainer(
      max_epochs=epochs,
      logger=neptune_logger,
      callbacks=[checkpoint_callback],
      gpus=1,
      # fast_dev_run=True,
  )

  model.cuda()
  trainer.fit(model)

  trainer.test(model)
  neptune_logger.experiment.stop()

  os.system("zip -r /content/models_checkpoint_"+ str(i) +'.zip '+'/content/models_'+str(i))
  files.download("/content/models_checkpoint_"+ str(i) + ".zip") 


NeptuneLogger will work in online mode


***********************RUNNING MODEL FOR ITERATION 3*********************
Device: cuda
https://ui.neptune.ai/leyldy/drugs-PAN-baseline/e/DRUGBASE-7


GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name         | Type              | Params
---------------------------------------------------
0 | criterion    | BCEWithLogitsLoss | 0     
1 | atom_encoder | AtomEncoder       | 5.5 K 
2 | conv1        | PANConv           | 16.9 K
3 | pool1        | PANPooling        | 518   
4 | conv2        | PANConv           | 131 K 
5 | pool2        | PANPooling        | 262   
6 | conv3        | PANConv           | 32.9 K
7 | lin1         | Linear            | 129   
---------------------------------------------------
187 K     Trainable params
0         Non-trainable params
187 K     Total params
0.750     Total estimated model params size (MB)
  0%|          | 0/3 [00:00<?, ?it/s]

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:01<00:00,  2.53it/s]


Extracting dataset/hiv.zip
Processing...
Loading necessary files...
This might take a while.


...

basemodel-nopoolend-posweight-meanloss


Saving latest checkpoint...



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6386855675080632, 'test_loss': 1.2973094456123584}
--------------------------------------------------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# Save model checkpoints again just to make sure
for i in range(num_iters):
  os.system("zip -r /content/models_checkpoint_dup_"+ str(i) +'.zip '+'/content/models_'+str(i))
  files.download("/content/models_checkpoint_dup_"+ str(i) + ".zip") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Seeing which model gave best validation results:

In [15]:
import neptune
myproj = neptune.init("leyldy/drugs-PAN-baseline", api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZDMyZTE4YmUtZTU1Yy00MDhhLTgyMzMtOTJiNzJjMTdkYWJkIn0=')
full_results = pd.DataFrame()
for myexp in myproj.get_experiments()[-num_iters:]:
  seed_result = myexp.get_numeric_channels_values('validation_loss', 'rocauc_eval', 'train_loss', 'rocauc_train')
  seed_result['exp_id'] = myexp.id
  seed_result['iter'] = seed_result['exp_id'].map(exp_id_dict)
  full_results = pd.concat([full_results, seed_result], axis=0)

full_results.rename(columns={'x': 'epoch'}, inplace=True)
print(full_results.isnull().sum())
full_results.to_csv('/content/full_train_val_results.csv', index=False)
files.download('/content/full_train_val_results.csv')

print(full_results.shape)
full_results.head()

epoch              0
rocauc_train       0
rocauc_eval        0
validation_loss    0
train_loss         0
exp_id             0
iter               0
dtype: int64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(750, 7)


,epoch,rocauc_train,rocauc_eval,validation_loss,train_loss,exp_id,iter
0,0.0,0.538741,0.631199,1.112725,1.657663,DRUGBASE-2,0
1,1.0,0.589510,0.646469,1.107210,1.482251,DRUGBASE-2,0
2,2.0,0.629160,0.630384,1.068710,1.390433,DRUGBASE-2,0
3,3.0,0.639964,0.645706,1.130972,1.393926,DRUGBASE-2,0
4,4.0,0.665126,0.690669,0.998687,1.348562,DRUGBASE-2,0


Method 1: Summarizing via choosing best iteration epoch per iteration:

In [17]:
idx = full_results.groupby(['exp_id'])['rocauc_eval'].transform(max) == full_results['rocauc_eval']

best_epoch_per_iter = full_results[idx]
best_epoch_per_iter

,epoch,rocauc_train,rocauc_eval,validation_loss,train_loss,exp_id,iter
80,80.0,0.748727,0.734975,0.958004,1.230091,DRUGBASE-2,0
26,26.0,0.697676,0.677951,1.018037,1.316323,DRUGBASE-3,1
105,105.0,0.715569,0.731687,1.016501,1.281574,DRUGBASE-4,2
125,125.0,0.706945,0.710878,0.992531,1.284142,DRUGBASE-7,3
124,124.0,0.685696,0.674983,1.021935,1.313290,DRUGBASE-8,4


In [27]:
from os import listdir
from os.path import isfile, join

test_results = {'iter': [], 'epoch': [], 'rocauc_test': [], 'test_loss': []}
model_paths = ['/content/models_' + str(i) for i in range(num_iters)]
test_trainer = Trainer(
      max_epochs=epochs,
      gpus=1,
      # fast_dev_run=True,
)
for i, mod_path in enumerate(model_paths):
  onlyfiles = [f for f in listdir(mod_path) if isfile(join(mod_path, f))]
  best_epoch = int(best_epoch_per_iter.loc[best_epoch_per_iter['iter']==i, 'epoch'])
  onlyfiles = [ckpt_nm for ckpt_nm in onlyfiles if 'epoch='+"{:02d}".format(best_epoch) in ckpt_nm]
  ckpt_nm = onlyfiles[0]
  print(i, ckpt_nm)

  ckpt_mod = LightningPAN.load_from_checkpoint(checkpoint_path=mod_path+"/"+ckpt_nm)
  ckpt_test_results = test_trainer.test(ckpt_mod)

  test_results['iter'] += [i]
  test_results['epoch'] += [best_epoch]
  test_results['rocauc_test'] += [ckpt_test_results[0]['rocauc_test']]
  test_results['test_loss'] += [ckpt_test_results[0]['test_loss']]

test_results_df = pd.DataFrame.from_dict(test_results).reset_index(drop=True)
id_to_exp_map = {val: key for key,val in exp_id_dict.items()}
test_results_df['exp_id'] = test_results_df['iter'].map(id_to_exp_map)

test_results_df.to_csv('/content/test_results.csv', index=False)
files.download('/content/test_results.csv')
test_results_df

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


0 PAN-epoch=80-validation_loss=0.9580-rocauc_eval=0.7350.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6813235095308909, 'test_loss': 1.2285363186489453}
--------------------------------------------------------------------------------
1 PAN-epoch=26-validation_loss=1.0180-rocauc_eval=0.6780.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.671653566117538, 'test_loss': 1.2484538320339087}
--------------------------------------------------------------------------------
2 PAN-epoch=105-validation_loss=0.9538-rocauc_eval=0.6880.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6564321443056065, 'test_loss': 1.2619828202507712}
--------------------------------------------------------------------------------
3 PAN-epoch=125-validation_loss=1.0575-rocauc_eval=0.6602.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6158915776666216, 'test_loss': 1.2984055858669858}
--------------------------------------------------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,iter,epoch,rocauc_test,test_loss,exp_id
0,0,80,0.681324,1.228536,DRUGBASE-2
1,1,26,0.671654,1.248454,DRUGBASE-3
2,2,105,0.656432,1.261983,DRUGBASE-4
3,3,125,0.615892,1.298406,DRUGBASE-7


In [31]:
final_results_df = test_results_df[['iter', 'rocauc_test', 'test_loss']].merge(best_epoch_per_iter, on='iter', how='outer')
final_results_df.to_csv('/content/final_results.csv', index=False)
files.download('/content/final_results.csv')
final_results_df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,iter,rocauc_test,test_loss,epoch,rocauc_train,rocauc_eval,validation_loss,train_loss,exp_id
0,0,0.681324,1.228536,80.0,0.748727,0.734975,0.958004,1.230091,DRUGBASE-2
1,1,0.671654,1.248454,26.0,0.697676,0.677951,1.018037,1.316323,DRUGBASE-3
2,2,0.656432,1.261983,105.0,0.715569,0.731687,1.016501,1.281574,DRUGBASE-4
3,3,0.615892,1.298406,125.0,0.706945,0.710878,0.992531,1.284142,DRUGBASE-7
4,4,NaN,NaN,124.0,0.685696,0.674983,1.021935,1.313290,DRUGBASE-8


Method 2: Summarizing per epoch across iterations

In [ ]:
# summ_results = full_results.groupby(['epoch']).mean().reset_index()
# summ_results.sort_values(['validation_loss'], ascending=True, inplace=True)
# summ_results.head()

,epoch,rocauc_eval,validation_loss
3,3.0,0.656087,34.415566
4,4.0,0.654051,34.751163
2,2.0,0.646194,35.530787
1,1.0,0.616151,38.533437
0,0.0,0.580398,41.788674


Finally, calculating test set results for this best validation performance epoch and downloading it:

In [ ]:
# from os import listdir
# from os.path import isfile, join

# best_epoch = int(summ_results['epoch'].values[0])

# test_results = {'rocauc_test': [], 'test_loss': []}
# model_paths = ['/content/models_' + str(i) for i in range(num_iters)]
# for mod_path in model_paths:
#   onlyfiles = [f for f in listdir(mod_path) if isfile(join(mod_path, f))]
#   onlyfiles = [ckpt_nm for ckpt_nm in onlyfiles if 'epoch='+"{:02d}".format(best_epoch) in ckpt_nm]
#   ckpt_nm = onlyfiles[0]
#   print(ckpt_nm)

#   ckpt_mod = LightningPAN.load_from_checkpoint(checkpoint_path=mod_path+"/"+ckpt_nm)
#   ckpt_test_results = trainer.test(ckpt_mod)

#   test_results['rocauc_test'] += [ckpt_test_results[0]['rocauc_test']]
#   test_results['test_loss'] += [ckpt_test_results[0]['test_loss']]

# test_results_df = pd.DataFrame.from_dict(test_results).reset_index()
# test_results_df.rename(columns={'index': 'iter'}, inplace=True)
# test_results_df.to_csv('/content/test_results.csv', index=False)
# files.download('/content/test_results.csv')
# test_results_df

PAN-epoch=03-validation_loss=33.18-rocauc_eval=0.69.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6376339828888159, 'test_loss': 42.10786324739456}
--------------------------------------------------------------------------------
PAN-epoch=03-validation_loss=34.33-rocauc_eval=0.66.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6550029934915699, 'test_loss': 40.97424465417862}
--------------------------------------------------------------------------------
PAN-epoch=03-validation_loss=35.74-rocauc_eval=0.62.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6086328434307344, 'test_loss': 42.3417871594429}
--------------------------------------------------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,iter,rocauc_test,test_loss
0,0,0.637634,42.107863
1,1,0.655003,40.974245
2,2,0.608633,42.341787


Command to keep colab running (run on browser console (i.e. chrome, cmd+shift+c))


In [ ]:
# function ClickConnect(){
#     console.log("Clicked on connect button"); 
#     document.querySelector("colab-connect-button").click()
# }
# setInterval(ClickConnect,60000)

# var startClickConnect = function startClickConnect(){
#     var clickConnect = function clickConnect(){
#         console.log("Connnect Clicked - Start");
#         document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
#         console.log("Connnect Clicked - End"); 
#     };

#     var intervalId = setInterval(clickConnect, 60000);

#     var stopClickConnectHandler = function stopClickConnect() {
#         console.log("Connnect Clicked Stopped - Start");
#         clearInterval(intervalId);
#         console.log("Connnect Clicked Stopped - End");
#     };

#     return stopClickConnectHandler;
# };

# var stopClickConnect = startClickConnect();

# stopClickConnect();


Exploratory Data Analysis:

In [ ]:
# # Importing dataset to do some quick EDA
# from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
# # Load the dataset 
# dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

# split_idx = dataset.get_idx_split()

# # Check task type
# print('Task type: {}'.format(dataset.task_type))

In [ ]:
# directed_g, undirected_g = 0, 0
# neg_graphs = []; pos_graphs = []
# num_edges_lst = [];
# num_nodes_lst = [];
# for i, curr_g in enumerate(dataset):
#   num_edges_lst.append(curr_g.num_edges)
#   num_nodes_lst.append(curr_g.num_nodes)
#   if curr_g.is_directed():
#     directed_g += 1
#   else:
#     undirected_g += 1
  
#   if curr_g.y.item() == 0:
#     neg_graphs.append(i)
#   elif curr_g.y.item() == 1:
#     pos_graphs.append(i)
#   else:
#     continue

In [ ]:
# print("Total number of graphs:", len(dataset))
# print("Number of classes:", dataset.num_classes)
# print("Dimension of node features:", dataset.num_node_features)
# print("Dimension of edge features:", dataset.num_edge_features)
# print("Number of positive labels:", dataset.data.y.sum().item())
# print("Number of negative labels:", dataset.data.y.shape[0] - dataset.data.y.sum().item())
# print("Min, Mean, Max # edges:", np.min(num_edges_lst), round(np.mean(num_edges_lst),2), np.max(num_edges_lst))
# print("Min, Mean, Max # nodes:", np.min(num_nodes_lst), round(np.mean(num_nodes_lst),2), np.max(num_nodes_lst))
# print("Number of directed graphs:", directed_g)
# print("Number of undirected graphs:", undirected_g)